## Load Libraries

In [1]:
import numpy as np
import MDAnalysis as md
import matplotlib.pyplot as plt
%matplotlib inline
from shapeGMMTorch import torch_sgmm, scripts, torch_align
import torch

## Load Trajectory

In [2]:
# point to parameter/topology file and trajectory file
parm_file = "../idp_14_19_vac.parm7"
traj_file = "idp_14_19_solv.nvt.stripped.dcd"
# create MD Universe
coord = md.Universe(parm_file,traj_file)
# make atom selection
sel = coord.select_atoms("all")
# create empty array to contain all selection coordinates
traj_data = np.empty((coord.trajectory.n_frames,sel.n_atoms,3),dtype=np.float64)
# read trajectory and store coordinates (minus COG)
for ts in coord.trajectory:
    traj_data[ts.frame] = sel.positions-sel.center_of_geometry()

## Align Trajectory to single average

In [3]:
usgmm1 = torch_sgmm.ShapeGMMTorch(1,covar_type="uniform",kabsch_thresh=1e-3,dtype=torch.float64,device=torch.device("cpu"),verbose=True)
aligned_traj_usgmm1 = usgmm1.fit(traj_data)

Number of frames being analyzed: 30000
Number of particles being analyzed: 109
Number of dimensions (must be 3): 3
Initializing clustering using method: random
Setting uniform frame weights
Weights from initial clusters in fit: [1.]
1 [1.] -327.945
2 [1.] -327.945


In [4]:
dtype = torch.float64
device = torch.device("cpu")
traj_data_tensor = torch.tensor(traj_data,dtype=dtype,device=device)
torch_align.torch_remove_center_of_geometry(traj_data_tensor,dtype=dtype,device=device)
aligned_traj_tensor, avg_tensor = torch_align.torch_iterative_align_uniform(traj_data_tensor,dtype=dtype,device=device,verbose=True)[:2]

-332.8261507481754
-328.04669961123693
-327.94715529870257
-327.9447539824331
-327.9446947302745


In [5]:
aligned_traj = aligned_traj_tensor.cpu().numpy()
print(np.linalg.norm(aligned_traj_usgmm1-aligned_traj))

1.1271106724880442


## Compute Covariance Matrix

In [6]:
disp = aligned_traj - avg_tensor.cpu().numpy()
disp = disp.reshape(-1,sel.n_atoms*3)
covar = np.dot(disp.T,disp)
covar /= (disp.shape[0]-1)

In [ ]:
# check covariance matrix has a null space of 6
e, v = np.linalg.eigh(covar)
print(e[:10])

In [7]:
np.savetxt("idp_14_19_uniform_covar.dat",covar)

In [8]:
np.savetxt("idp_14_19_uniform_avg.dat", avg_tensor.cpu().numpy())

In [ ]:
#plot covariance matrix

n_nodes = covar.shape[0]
node_range = range(nNodes+1)
major_ticks = n_nodes//10
minoe_ticks = n_nodes//100
fig, ax = plt.subplots()
ax.tick_params(which='major',length=6,width=2)
ax.tick_params(which='minor',length=3,width=1)
ax.xaxis.set_minor_locator(MultipleLocator(minor_ticks))
ax.xaxis.set_major_locator(MultipleLocator(major_ticks))
ax.yaxis.set_minor_locator(MultipleLocator(minor_ticks))
ax.yaxis.set_major_locator(MultipleLocator(major_ticks))
# make vrange symmetric about zero
v_min = min(np.amin(covar),-np.amax(covar))
v_max = max(np.amax(covar),-np.amin(covar))
temp = plt.pcolormesh(node_range,node_range,square_matrix,cmap=plotting_cmap,vmin=v_min,vmax=v_max)

cb1 = plt.colorbar()
cb1.set_label(r'%s'%(cbar_label))

xlabels = [str(int(x)+1) for x in temp.axes.get_xticks()[:]]
ylabels = [str(int(y)+1) for y in temp.axes.get_yticks()[:]]
temp.axes.set_xticks(temp.axes.get_xticks(minor=True)[:]+0.5,minor=True)
temp.axes.set_xticks(temp.axes.get_xticks()[:]+0.5)
temp.axes.set_yticks(temp.axes.get_yticks(minor=True)[:]+0.5,minor=True)
temp.axes.set_yticks(temp.axes.get_yticks()[:]+0.5)
temp.axes.set_xticklabels(xlabels)
temp.axes.set_yticklabels(ylabels)

plt.xlim((0,nNodes))
plt.ylim((0,nNodes))
plt.xlabel("Residue Index",size=14)
plt.ylabel("Residue Index",size=14)
ax.set_aspect('equal')
plt.tight_layout()
plt.plot(np.arange(0,nNodes,0.01),np.arange(0,nNodes,0.01),'--',lw=2,color='k')
plt.show();

## Perform cluster scan

In [9]:
# define cluster array
cluster_array = np.arange(1,6,1).astype(int)
print(cluster_array)

[1 2 3 4 5]


In [ ]:
# perform cluster scan
train, cv = scripts.cross_validate_cluster_scan(traj_data, 25000, covar_type="kronecker", cluster_array = cluster_array, n_training_sets=3, n_attempts = 5, dtype=dtype, device=device)

Number of frames to train each model: 25000
Number of frames to predict each model: 5000
Number of training sets: 3
Number of clusters: 5
Number of attempts per set/cluster: 5
   Training Set      N Clusters         Attempt  Log Like per Frame    CPU Time (s)
------------------------------------------------------------------------------------
              1               1               1             405.849         227.648
              1               2               1             424.638         303.871
              1               2               2             424.625         306.041
              1               2               3             424.575         336.530
              1               2               4             424.591         314.221
              1               2               5             424.599         382.668
              1               3               1             428.226         308.069
              1               3               2             428.184

In [ ]:
# shapeGMMTorch plots has a built-in plot function for this type of plot
from shapeGMMTorch import plots
# also need to load matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
# create figure
fig, ax = plt.subplots(1,1,figsize=(4,4), dpi= 120, facecolor='w', edgecolor='k',sharex=True,sharey=True)
plots.plot_log_likelihood_with_dd(ax,cluster_array,train,cv,fontsize=12,ylabel1=True,ylabel2=True,xlabel=True)
# show plot
plt.tight_layout()
plt.show()